Dataset

In [1]:
!pip install roboflow


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="CBnLR8GsW5vPHMZbsyXO")
project = rf.workspace("skripsi-1qzlz").project("vehicle-detection-bvxpr")
version = project.version(2)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Vehicle-Detection-2 in yolov8:: 100%|██████████| 1194/1194 [00:00<00:00, 3913.01it/s]


Augmentation

In [1]:
!pip install albumentations opencv-python-headless


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import albumentations as A
from albumentations.core.composition import OneOf
from albumentations import HorizontalFlip, RandomBrightnessContrast, ShiftScaleRotate
import cv2
import os

#Define Aug
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5)
])

In [3]:
#image path
image_dir = "Vehicle-Detection-2/train/images"
label_dir = "Vehicle-Detection-2/train/labels"

aug_image_dir = "Vehicle-Detection-2/train/aug_images"
aug_label_dir = "Vehicle-Detection-2/train/aug_labels"

#Create Dir for aug
os.makedirs(aug_image_dir, exist_ok=True)
os.makedirs(aug_label_dir, exist_ok=True)

In [4]:
#apply aug
for img_file in os.listdir(image_dir):
    if img_file.endswith(".jpg") or img_file.endswith(".png"):
        img_path = os.path.join(image_dir, img_file)
        image = cv2.imread(img_path)
        height, width = image.shape[:2]

        label_file = img_file.replace(".jpg", ".txt").replace(".png",".txt")
        label_path = os.path.join(label_dir, label_file)

        if os.path.exists(label_path):
            with open(label_path, "r") as f:
                annotations = f.readlines()

            augmented = transform(image=image)
            augmented_image = augmented["image"]

            aug_img_path = os.path.join(aug_image_dir, "aug_" + img_file)
            cv2.imwrite(aug_img_path, augmented_image)

In [5]:
#adjust annotations 
with open(os.path.join(aug_label_dir, "aug_" + label_file), "w") as f:
    for line in annotations:
        class_id, x_center, y_center, w, h = map(float, line.split())
        x_center *= width
        y_center *= height
        #flip img
        x_center /= width
        y_center /= height
        w /= width
        h /= height
        f.write(f"{int(class_id)} {x_center} {y_center} {w} {h}\n")